In [1]:
import numpy as np

In [2]:
X_fea0 = np.array([1 for i in range(200)]).reshape(-1,1)
X_fea1 = np.array([1*i*np.random.normal(1,0.1) for i in range(200)]).reshape(-1,1)
X_fea2 = np.array([2*i*np.random.normal(0,5) for i in range(200)]).reshape(-1,1)
X_fea3 = np.array([3*i*np.random.normal(10,1) for i in range(200)]).reshape(-1,1)
X_fea4 = np.array([4*i*np.random.normal(2,0.01) for i in range(200)]).reshape(-1,1)
X_fea5 = np.array([5*i*np.random.normal(0,0.1) for i in range(200)]).reshape(-1,1)
X_fea6 = np.array([6*i*np.random.normal(50,5) for i in range(200)]).reshape(-1,1)
X_fea7 = np.array([7*i*np.random.normal(5,5) for i in range(200)]).reshape(-1,1)

# データセットを準備
X = np.concatenate([X_fea0,X_fea1,X_fea2,X_fea3,X_fea4,X_fea5,X_fea6,X_fea7],axis=1)
X_sklearn = np.concatenate([X_fea1,X_fea2,X_fea3,X_fea4,X_fea5,X_fea6,X_fea7],axis=1)

y = np.array([2*(i) for i in range(200)])

print(f'元のデータの特徴量数：{X_sklearn.shape[1]}')

元のデータの特徴量数：7


### PCAの理論




- PCAは元データの線形結合で得られる要約変数(=主成分, z)の分散を最大化する重みを算出する


$$
max \ \  \sigma_z^{2}\\
ただし、\sigma_z^{2} = \frac{1}{N} \sum_i^{n}{z_i^{2}} \ \ (zは標準化済み)
$$

$$
分散部分を式変形すると以下の形になる\\
\sigma_{z}^{2} = \textbf{u}^{T} \sigma^{2} \textbf{u}
$$

$$
ここで\\
\mathbf{\sum} = \sigma^{2},\ \ \  \mathbf{u^{T}u} = 1\\
と制約を与え、ラグランジュの未定乗数法で最適解を求める
$$

$$
\mathbf{L} = \mathbf{u^{T}\sum u} - \lambda(\mathbf{uu^{T}} - 1)\\
$$

$$
\frac{\partial{\mathbf{L}}}{\partial{\mathbf{u}}} = (\mathbf{\sum} + \mathbf{\sum}^{T})\mathbf{u} - 2\lambda\mathbf{u} = 0
$$

$$
\mathbf{\sum}は対称行列なので最終的に以下にまとめられる
$$

$$
\mathbf{\sum}\mathbf{u} = \lambda\mathbf{u}
$$

$$
上の式の形は、\mathbf{A}\mathbf{u} = \lambda\mathbf{u}であり、固有値問題と同じ形となる
$$

$$
つまりPCAは、\mathbf{\sum} \ (分散共分散行列)の固有値問題を解くことと同義である\\
また\mathbf{u}は固有ベクトルであり、各主成分のローディングである\\
そして\lambdaは各主成分の分散そのものであり、これを分散の総和で割ったものを寄与率と定義している
$$



In [3]:
# 1. データを標準化する

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_scale = scaler.fit_transform(X_sklearn)

In [4]:
# 2. 線形結合データ(主成分)の分散を最大化する重みを決定
# -> 線形結合で作成した軸に射影したデータの分散が最大となる重みを算出

# 分散共分散行列における固有値問題に帰結
VCM = (1/X_scale.shape[0])*(X_scale.T@X_scale)

# 固有値問題を解く
import numpy.linalg as LA
temp = LA.eig(VCM)

In [5]:
koyuti = temp[0]
koyuvector = temp[1]

In [6]:
# 寄与率 (各主成分の分散(固有値)を全主成分の分散の総和で除した値)
print(koyuti/koyuti.sum())

[0.60282489 0.16693998 0.12990844 0.0894286  0.0013436  0.00526441
 0.00429008]


In [7]:
print(koyuvector)

[[-0.47573356 -0.05068049 -0.07072959 -0.14671909 -0.24150167 -0.63923753
   0.52690817]
 [-0.06623288  0.71052167 -0.5497126   0.4317111  -0.01195665 -0.04519099
  -0.00534368]
 [-0.47754486  0.00355348 -0.07995256 -0.10579395 -0.28770962  0.75655563
   0.31496072]
 [-0.48175694 -0.01118862 -0.07015693 -0.11919036  0.86318895  0.02305836
  -0.05504435]
 [ 0.01874031 -0.69573566 -0.62465321  0.3530361  -0.00249023  0.00848936
  -0.02638759]
 [-0.47825087 -0.01391006 -0.04569273 -0.14102369 -0.33708086 -0.12691094
  -0.78700522]
 [-0.28297854 -0.09061478  0.53779133  0.78878271 -0.00568277 -0.01570275
   0.00596361]]


In [8]:
# sklearnのPCAを実装

from sklearn.decomposition import PCA

pca = PCA()

results = pca.fit(X_scale)

print('各主成分のLoadingは以下')
print(pca.components_.T)

print()
print('主成分ごとの寄与率')
print(pca.explained_variance_ratio_)

# sklearnのPCAはSVDで行っているため、結果が少し異なる

各主成分のLoadingは以下
[[ 0.47573356  0.05068049  0.07072959 -0.14671909 -0.63923753 -0.52690817
   0.24150167]
 [ 0.06623288 -0.71052167  0.5497126   0.4317111  -0.04519099  0.00534368
   0.01195665]
 [ 0.47754486 -0.00355348  0.07995256 -0.10579395  0.75655563 -0.31496072
   0.28770962]
 [ 0.48175694  0.01118862  0.07015693 -0.11919036  0.02305836  0.05504435
  -0.86318895]
 [-0.01874031  0.69573566  0.62465321  0.3530361   0.00848936  0.02638759
   0.00249023]
 [ 0.47825087  0.01391006  0.04569273 -0.14102369 -0.12691094  0.78700522
   0.33708086]
 [ 0.28297854  0.09061478 -0.53779133  0.78878271 -0.01570275 -0.00596361
   0.00568277]]

主成分ごとの寄与率
[0.60282489 0.16693998 0.12990844 0.0894286  0.00526441 0.00429008
 0.0013436 ]
